In [ ]:
import os
import opyf
import cv2
import subprocess

In [ ]:
#to trim the video-Note some files have heigh 862 pixels so u may have to change the ffmpeg commands for some videos according to the region of interest
def trim_and_crop_videos(directory_folder):
    # Change the current working directory to the common directory folder
    os.chdir(directory_folder)

    input_folder = "/home/arutkeerthi/SolarProject/HV_NN/Videos"
    output_folder = "/home/arutkeerthi/SolarProject/HV_NN/Cutvideos"

    # Loop through each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".mp4"):
            # Construct the full paths for input and output files
            input_file = os.path.join(input_folder, filename)
            output_filename = os.path.join(output_folder, f"C{filename}")

            # FFmpeg command to trim and crop the video
            ffmpeg_command = [
                "ffmpeg",
                "-i", input_file,
                "-filter:v", "crop=1250:934:360:0",
                output_filename
            ]

            # Run the FFmpeg command
            subprocess.run(ffmpeg_command)

# Example usage:
directory_folder = "/home/arutkeerthi/SolarProject/HV_NN"
trim_and_crop_videos(directory_folder)

In [ ]:
#To split videos into frames
def split_videos_to_frames(input_folder, output_root_folder):

    # Loop through each video file in the input folder
    for video_file in os.listdir(input_folder):
        if video_file.endswith(".mp4"):
            # Construct the full path for the input video
            input_video = os.path.join(input_folder, video_file)

            # Create an output folder with the same name as the video (without extension)
            video_name_no_ext = os.path.splitext(video_file)[0]
            output_folder = os.path.join(output_root_folder, video_name_no_ext)
            os.makedirs(output_folder, exist_ok=True)

            # FFmpeg command to split the video into frames
            ffmpeg_command = [
                "ffmpeg",
                "-i", input_video,
                "-vf", "fps=5",  # Adjust the frame rate as needed
                os.path.join(output_folder, "%000d.png")
            ]

            # Run the FFmpeg command
            subprocess.run(ffmpeg_command)

# Example usage:
input_folder = "/home/arutkeerthi/SolarProject/HV_NN/Cutvideos"
output_root_folder = "/home/arutkeerthi/SolarProject/HV_NN/Images"
split_videos_to_frames(input_folder, output_root_folder)

In [ ]:
#bilateral filter on all frames to reduce noise while not smoothening too much
def bilateral_filter_images(input_root, output_root):
    # Walk through the input root directory
    for root, _, files in os.walk(input_root):
        # Create an equivalent folder structure in the output root directory
        rel_path = os.path.relpath(root, input_root)
        output_folder = os.path.join(output_root, rel_path)
        os.makedirs(output_folder, exist_ok=True)

        # Process each image file in the current folder
        for filename in files:
            if filename.endswith('.png'):
                # Read the image
                img_path = os.path.join(root, filename)
                img = cv2.imread(img_path)

                # Apply bilateral filter with d = 50, sigmaColor = 75, sigmaSpace = 75
                bilateral = cv2.bilateralFilter(img, 50, 75, 75)

                # Save the filtered image to the corresponding output folder with the same filename
                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, bilateral)

    print("Bilateral filtering completed for all images.")

# Example usage:
input_root = '/home/arutkeerthi/SolarProject/HV_NN/Images'
output_root = '/home/arutkeerthi/SolarProject/HV_NN/Filteredimages'
bilateral_filter_images(input_root, output_root)

In [ ]:
def analyze_videos_in_subfolders(root_folder, image_root_folder):
    # Define output_csv_filename outside the loop
    output_csv_filename = ""

    # Walk through the root folder
    for root, dirs, _ in os.walk(root_folder):
        for folder_name in dirs:
            folder_path = os.path.join(root, folder_name)
            output_csv_filename = folder_name + '.csv'

            # Calculate Ntot as the number of files in the folder minus two
            file_list = os.listdir(folder_path)
            Ntot = max(len(file_list) - 1, 0)  # Ensure Ntot is at least 0

            # Initialize video analysis for the current subfolder
            video = opyf.frameSequenceAnalyzer(folder_path)
            video.set_vecTime(Ntot=Ntot, step=1, shift=1, starting_frame=0)
            video.set_goodFeaturesToTrackParams(maxCorners=4000, qualityLevel=0.002)
            video.set_opticalFlowParams(maxLevel=3, winSize=(8, 8))
            video.set_vlim([0, 250])
            video.set_filtersParams(wayBackGoodFlag=10, RadiusF=5, maxDevInRadius=2)

            # Create a subfolder in the image_root_folder with the same name as the video
            image_subfolder = os.path.join(image_root_folder, folder_name)
            os.makedirs(image_subfolder, exist_ok=True)
            
            video.extractGoodFeaturesAndDisplacements(display='quiver', displayColor=True, width=0.003,
                                                     saveImgPath=image_subfolder)

            # Use output_csv_filename here
            output_csv_path = os.path.join(folder_path, output_csv_filename)
            video.writeGoodFeaturesPositionsAndDisplacements(fileFormat='csv', outFolder='/home/arutkeerthi/SolarProject/HV_NN/csv',
                                                            filename=output_csv_filename)

# Example usage:
root_folder = '/home/arutkeerthi/SolarProject/HV_NN/Filteredimages'
image_root_folder = '/home/arutkeerthi/SolarProject/HV_NN/Opyflow_images'  # Change this to your desired image root folder
analyze_videos_in_subfolders(root_folder, image_root_folder)